In [1]:
#run only once
#!pip install node2vec

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: distro-info 0.23ubuntu1 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: python-debian 0.1.36ubuntu1 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of python-debian or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [66]:
#import libraries
import pandas as pd
import string as st
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import sys
from node2vec import Node2Vec
import pickle

In [67]:
#import data
data = pd.read_csv("IMDB-Movie-Data.csv")
data.head()

,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,333.13,76.0
1,2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820,126.46,65.0
2,3,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,157606,138.12,62.0
3,4,Sing,"Animation,Comedy,Family","In a city of humanoid animals, a hustling thea...",Christophe Lourdelet,"Matthew McConaughey,Reese Witherspoon, Seth Ma...",2016,108,7.2,60545,270.32,59.0
4,5,Suicide Squad,"Action,Adventure,Fantasy",A secret government agency recruits some of th...,David Ayer,"Will Smith, Jared Leto, Margot Robbie, Viola D...",2016,123,6.2,393727,325.02,40.0


In [68]:
# Data Pre-processing Functions
# Remove Punctuations
def remove_punctuation(text):
    return ("".join([ch.lower() for ch in text if ch not in st.punctuation]))

def preprocess(word):
    if word[0] == ' ':
        word = word[1:]
    return remove_punctuation(word.lower())

#data['Title'] = remove_punctuation(data['Title'])

In [74]:
#function to create graph nodes using lists if node 1, node 2, relation

def getGraphNodes(node1, node2, relation):
    temp1 = []
    temp2 = []
    for i in range(0, len(node1)):
        if isinstance(node2[i], (int, np.integer)):
            # For year
            temp1.append(remove_punctuation(node1[i]))
            temp2.append(str(node2[i]))
        else :
            # For director, actor, genre
            for mov in node2[i].split(','):
                temp1.append(preprocess(node1[i]))
                temp2.append(preprocess(mov))
            
    graphNodesNew = pd.DataFrame(list(zip(temp1, temp2, [relation]*len(temp1) )))
    graphNodesNew.columns = ['Node 1', 'Node 2', 'Relation']
    return graphNodesNew

graphNodes = pd.DataFrame()

#create graph nodes for Director-movie relation
graphNodes = graphNodes.append(getGraphNodes(data['Title'], data['Director'], 'directing'))

#create graph nodes for Actor-movie relation
graphNodes = graphNodes.append(getGraphNodes(data['Title'], data['Actors'], 'acting'))

#create graph nodes for Genre-movie relation
graphNodes = graphNodes.append(getGraphNodes(data['Title'], data['Genre'], 'genre'))

#create graph nodes for Year-movie relation
graphNodes = graphNodes.append(getGraphNodes(data['Title'], data['Year'], 'released'))


/tmp/ipykernel_46941/3076032495.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  graphNodes = graphNodes.append(getGraphNodes(data['Title'], data['Director'], 'directing'))
/tmp/ipykernel_46941/3076032495.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  graphNodes = graphNodes.append(getGraphNodes(data['Title'], data['Actors'], 'acting'))
/tmp/ipykernel_46941/3076032495.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  graphNodes = graphNodes.append(getGraphNodes(data['Title'], data['Genre'], 'genre'))
/tmp/ipykernel_46941/3076032495.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  graphNodes = graphNodes.append(getGraphN

In [75]:
graphNodes

,Node 1,Node 2,Relation
0,guardians of the galaxy,james gunn,directing
1,prometheus,ridley scott,directing
2,split,m night shyamalan,directing
3,sing,christophe lourdelet,directing
4,suicide squad,david ayer,directing
...,...,...,...
995,secret in their eyes,2015,released
996,hostel part ii,2007,released
997,step up 2 the streets,2008,released
998,search party,2014,released


In [76]:
# create a undirected-graph from a dataframe
G = nx.from_pandas_edgelist(graphNodes, "Node 1", "Node 2", edge_attr=True, create_using=nx.MultiDiGraph())
G = G.to_undirected()

In [77]:
# Generate embeddings for graph nodes using Node2Vec
graph = G
# Initialize Node2Vec with the graph
node2vec = Node2Vec(graph, dimensions=16, walk_length=30, num_walks=200, workers=4)
# Learn embeddings
embeddings = node2vec.fit(window=3, min_count=1, batch_words=4)

Computing transition probabilities:   0%|          | 0/3622 [00:00<?, ?it/s]

/usr/lib/python3/dist-packages/paramiko/transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,
Generating walks (CPU: 1):   0%|          | 0/50 [00:00<?, ?it/s]/usr/lib/python3/dist-packages/paramiko/transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,
Generating walks (CPU: 2):   4%|▍         | 2/50 [00:00<00:11,  4.24it/s]/usr/lib/python3/dist-packages/paramiko/transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,
Generating walks (CPU: 2):   6%|▌         | 3/50 [00:00<00:15,  2.96it/s]/usr/lib/python3/dist-packages/paramiko/transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,
Generating walks (CPU: 4): 100%|██████████| 50/50 [00:37<00:00,  1.33it/s]

In [78]:
# Generate pkl files so you dont need to create graphs and embeddings again

with open('graph.pkl', 'wb') as f:
    pickle.dump(graph, f)
    
with open('embeddings.pkl', 'wb') as f:
    pickle.dump(embeddings, f)

### Just Experimentation

In [88]:
similar_nodes = embeddings.wv.most_similar('sanjay dutt', topn=20)
for n in similar_nodes:
    print(n[0])

boman irani
anushka sharma
pk
rajkumar hirani
aamir khan
madhavan
sharman joshi
3 idiots
sex tape
mona singh
ellie kemper
jake kasdan
rob corddry
lyndsy fonseca
bruce campbell
will slocombe
tommy dewey
amma asante
the escort
michael doneger


In [43]:
emb = {node: embeddings.wv[str(node)] for node in G.nodes()}

In [44]:
emb.keys()

dict_keys(['Guardians of the Galaxy', 'James Gunn', 'Prometheus', 'Ridley Scott', 'Split', 'M. Night Shyamalan', 'Sing', 'Christophe Lourdelet', 'Suicide Squad', 'David Ayer', 'The Great Wall', 'Yimou Zhang', 'La La Land', 'Damien Chazelle', 'Mindhorn', 'Sean Foley', 'The Lost City of Z', 'James Gray', 'Passengers', 'Morten Tyldum', 'Fantastic Beasts and Where to Find Them', 'David Yates', 'Hidden Figures', 'Theodore Melfi', 'Rogue One', 'Gareth Edwards', 'Moana', 'Ron Clements', 'Colossal', 'Nacho Vigalondo', 'The Secret Life of Pets', 'Chris Renaud', 'Hacksaw Ridge', 'Mel Gibson', 'Jason Bourne', 'Paul Greengrass', 'Lion', 'Garth Davis', 'Arrival', 'Denis Villeneuve', 'Gold', 'Stephen Gaghan', 'Manchester by the Sea', 'Kenneth Lonergan', 'Hounds of Love', 'Ben Young', 'Trolls', 'Walt Dohrn', 'Independence Day: Resurgence', 'Roland Emmerich', 'Paris pieds nus', 'Dominique Abel', 'Bahubali: The Beginning', 'S.S. Rajamouli', 'Dead Awake', 'Phillip Guzman', 'Bad Moms', 'Jon Lucas', "Assa

In [45]:
len(graph.nodes())

3623

In [31]:
edges_for_node = graph.edges('Anushka Shetty', data=True)
print(edges_for_node)
for edge in edges_for_node:
    print(edge)

[]
